In [ ]:
import pandas as pd
import re
import numpy as np
from collections import Counter
from rapidfuzz import fuzz, process

# Load the Tata CSV file
tata_data = pd.read_csv('/home/bipin/Documents/kotak/car_model/manufacturer_data/TATA.csv')
print(f"Loaded {len(tata_data)} records from TATA.csv")
tata_data.head()

# Function to clean and normalize model names
def clean_model(text):
    if pd.isna(text) or text is None:
        return ""
    # Convert to string
    text = str(text).upper()
    # Remove common prefixes and company names
    text = re.sub(r'TATA\s+MOTORS\s+LTD,?\s*', '', text)
    text = re.sub(r'TATA\s+MOTORS\s+LIMITED,?\s*', '', text)
    text = re.sub(r'M/S\.\s+TATA\s+MOTORS\s+LTD,?\s*', '', text)
    text = re.sub(r'TATA\s+ENGINEERING\s+AND\s+LOCOMOTIVE\s+CO\.,?\s*', '', text)
    text = re.sub(r'TATA\s+ENGG\.\s+AND\s+LOCO\s+CO\.,?\s*', '', text)
    text = re.sub(r'TELCO\s+LTD\.?,?\s*', '', text)
    text = re.sub(r'TATA\s+', '', text)
    text = re.sub(r'\"', '', text)
    
    # Remove special characters and extra spaces
    text = re.sub(r'[^\w\s\./\-]', ' ', text)  # Keep ., /, and -
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Create a list of cleaned model names
tata_data['clean_model'] = tata_data['rc_maker_model'].apply(clean_model)

# Filter out too generic or empty model names
generic_terms = ['LTD', 'BS', 'IV', 'III', 'BS-IV', 'BS-III', 'BSIV', 'BSIII', 'BS6', 'BSVI', 'LIMITED', 'PVT']
filtered_models = tata_data['clean_model'].apply(
    lambda x: x if x and all(term not in x.split() for term in generic_terms) else np.nan
)

# Count model name frequencies
model_counts = Counter([m for models in filtered_models.dropna() for m in models.split()])
model_counts = {k: v for k, v in model_counts.items() if v >= 10}  # Keep only frequent terms

# Print top model keywords by frequency
print("Top model keywords by frequency:")
sorted_counts = sorted(model_counts.items(), key=lambda x: x[1], reverse=True)
for model, count in sorted_counts[:30]:
    print(f"{model}: {count}")

# Define primary Tata model keywords
tata_model_keywords = [
    # Passenger cars - Sedans & Hatchbacks
    "INDICA", "INDIGO", "VISTA", "MANZA", "ZEST", "BOLT", "TIAGO", "TIGOR", "ALTROZ", "NANO",
    
    # SUVs & Crossovers
    "SAFARI", "SUMO", "NEXON", "HARRIER", "HEXA", "ARIA", "PUNCH", "STORME", "GRANDE", "GRAVITAS",

    # Commercial Vehicles - Small
    "ACE", "INTRA", "MAGIC", "WINGER", "VENTURE", "IRIS", "SUPER", "XENON", "YODHA", "ZIP",
    
    # Commercial Vehicles - Trucks
    "SIGNA", "PRIMA", "ULTRA", "LPT", "LPS", "LPK", "LPO", "SFC", "SK", "SE",
    
    # Variant & Sub-model Specifiers
    "HT", "EX", "EX2", "GX", "LX", "VX", "XE", "XM", "XT", "XZ", "XZ+", "ELS", "GLX", "GLS", 
    
    # Engine Variants
    "DICOR", "QJET", "QUADRAJET", "SAFIRE", "REVOTRON", "REVORTRQ", "VARICOR", "KRYOTEC", "XETA",
    
    # Special Variants & Trims
    "GOLD", "GRAND", "VICTA", "VISTA", "MAGIC", "STARBUS", "NRG", "FACELIFT", "DARK",
    
    # Vehicle Type Designations
    "PICKUP", "BUS", "TRUCK", "TIPPER", "CHASSIS", "COWL", "MINIBUS", "STARBUS", "CITYRIDE",
    
    # Engine Sizes & Specifications
    "1.1", "1.2", "1.3", "1.4", "1.5", "2.0", "2.2", "3.0",
    
    # Drive Types
    "4X2", "4X4", "2WD", "4WD",
    
    # Common Model-Specific Combinations
    "INDICA V2", "INDIGO MARINA", "INDIGO CS", "INDIGO ECS", "INDIGO MANZA",
    "SAFARI STORME", "SAFARI DICOR", "SUMO GOLD", "SUMO VICTA", "SUMO GRANDE",
    "TIAGO NRG", "NEXON EV", "ACE GOLD", "ACE MEGA", "ACE MAGIC", "ACE ZIP",
    
    # Common Commercial Vehicle Model Numbers
    "407", "709", "909", "1109", "1512", "1613", "2213", "2516", "2518", "3118", "3518", "4018", "4923"
]

# Create alias map for common misspellings and variations
tata_alias_map = {
    # Passenger Car Aliases
    "INDICAB": "INDICA",
    "INDICAV2": "INDICA V2",
    "INDCA": "INDICA",
    "INDICO": "INDIGO",
    "INDGA": "INDIGO",
    "INDGIO": "INDIGO",
    "INDIGOCS": "INDIGO CS",
    "INDIGOECS": "INDIGO ECS",
    "INDIGOMANZA": "INDIGO MANZA",
    "INDIGOM": "INDIGO MANZA",
    "MANZ": "MANZA",
    "VIST": "VISTA",
    "VSTA": "VISTA",
    "ZST": "ZEST",
    "ZEDT": "ZEST",
    "BLT": "BOLT",
    "TIGO": "TIAGO",
    "TIAG": "TIAGO",
    "TIGR": "TIGOR",
    "TIGORE": "TIGOR",
    "ALTRZ": "ALTROZ",
    "ALTRI": "ALTROZ",
    "NENO": "NANO",
    "NAON": "NANO",
    
    # SUV & Crossover Aliases
    "SAFRI": "SAFARI",
    "SAFFARI": "SAFARI",
    "SAFARISTORME": "SAFARI STORME",
    "SAFARIST": "SAFARI STORME",
    "SAFARIMINOR": "SAFARI",
    "SAFARIDICOR": "SAFARI DICOR",
    "STOR": "STORME",
    "SUMOVICTA": "SUMO VICTA",
    "SUMOGOLD": "SUMO GOLD",
    "SUMOGRANDE": "SUMO GRANDE",
    "SUMOGRAND": "SUMO GRAND",
    "NEXN": "NEXON",
    "NEXONE": "NEXON",
    "NEXONEV": "NEXON EV",
    "HARR": "HARRIER",
    "HEX": "HEXA",
    "HXA": "HEXA",
    "HEAX": "HEXA",
    "ARA": "ARIA",
    "PUNSH": "PUNCH",
    "PNCH": "PUNCH",
    
    # Commercial Vehicle Aliases
    "ACEHT": "ACE HT",
    "ACEGOLD": "ACE GOLD",
    "ACEZIP": "ACE ZIP",
    "ACEX": "ACE EX",
    "ACEMAGIC": "ACE MAGIC",
    "INTR": "INTRA",
    "MAG": "MAGIC",
    "MAGICEXPRESS": "MAGIC EXPRESS",
    "MAGICIRIS": "MAGIC IRIS",
    "WNGR": "WINGER",
    "WNGER": "WINGER",
    "VENTUR": "VENTURE",
    "XENN": "XENON",
    "XENONDICOR": "XENON DICOR",
    "XENONYODHA": "XENON YODHA",
    "YODH": "YODHA",
    "SUPERACE": "SUPER ACE",
    "SUPRACE": "SUPER ACE",
    "SIGNA3118": "SIGNA 3118",
    "SIGNA4018": "SIGNA 4018",
    "SIGNA4923": "SIGNA 4923",
    "PRIMA4018": "PRIMA 4018",
    "PRIMA4923": "PRIMA 4923",
    "LPT407": "LPT 407",
    "LPT709": "LPT 709",
    "LPT909": "LPT 909",
    "LPT1109": "LPT 1109",
    "LPT1613": "LPT 1613",
    "LPT2518": "LPT 2518",
    "LPT3118": "LPT 3118",
    "LPS4018": "LPS 4018",
    "LPK2518": "LPK 2518",
    "SFC407": "SFC 407",
    "SE1613": "SE 1613",
    
    # Trim & Variant Aliases
    "XZ PLUS": "XZ+",
    "XT PLUS": "XT+",
    "XM PLUS": "XM+",
    "XE PLUS": "XE+",
    "XZP": "XZ+",
    "XTP": "XT+",
    "XMP": "XM+",
    "XEP": "XE+",
    "XZPLUSO": "XZ+O",
    "XZPLUSDARK": "XZ+ DARK",
    
    # Engine & Specification Aliases
    "QDRJET": "QUADRAJET",
    "QJET": "QUADRAJET",
    "Q JET": "QUADRAJET",
    "QJADRAJET": "QUADRAJET",
    "QUARDRAJET": "QUADRAJET",
    "QUARDAJET": "QUADRAJET",
    "QUDRAJET": "QUADRAJET",
    "RVTRN": "REVOTRON",
    "RTN": "REVOTRON",
    "REVTRN": "REVOTRON",
    "REVRTRQ": "REVORTRQ",
    "RTQ": "REVORTRQ",
    "KRYOTC": "KRYOTEC",
    "KRYTEC": "KRYOTEC",
    "DICR": "DICOR",
    "XETTA": "XETA",
    "SAFFIRE": "SAFIRE",
    "SFIRE": "SAFIRE",
    "VARICR": "VARICOR",
    
    # Common Model-Specific Combinations
    "SUMO GOLD GX": "SUMO GOLD",
    "SUMO GOLD EX": "SUMO GOLD",
    "SUMO GOLD LX": "SUMO GOLD",
    "SUMO VICTA GX": "SUMO VICTA",
    "SUMO VICTA EX": "SUMO VICTA",
    "SUMO VICTA LX": "SUMO VICTA",
    "SUMO GRANDE GX": "SUMO GRANDE",
    "SUMO GRANDE EX": "SUMO GRANDE",
    "SUMO GRANDE LX": "SUMO GRANDE",
    
    "SAFARI STORME VX": "SAFARI STORME",
    "SAFARI STORME EX": "SAFARI STORME",
    "SAFARI STORME LX": "SAFARI STORME",
    
    "SAFARI DICOR VX": "SAFARI DICOR",
    "SAFARI DICOR EX": "SAFARI DICOR",
    "SAFARI DICOR LX": "SAFARI DICOR",
    
    "NEXON XZ": "NEXON",
    "NEXON XM": "NEXON",
    "NEXON XE": "NEXON",
    "NEXON XT": "NEXON",
    
    # Commercial Vehicle Combinations
    "LPT 407 EX2": "LPT 407",
    "LPT 709 EX2": "LPT 709",
    "LPT 909 EX2": "LPT 909",
    "LPT 1109 EX2": "LPT 1109",
    
    # Non-Tata vehicles in the data
    "FIAT PALIO": "NOT TATA",
    "FIAT LINEA": "NOT TATA",
    "LAND ROVER": "JAGUAR LAND ROVER",
    "RANGE ROVER": "JAGUAR LAND ROVER",
    "JAGUAR": "JAGUAR LAND ROVER"
}

# Function to normalize text
def normalize(text):
    # Handle NaN/None values or any non-string types
    if pd.isna(text) or text is None:
        return ""
    # Convert to string if it's not already a string
    if not isinstance(text, str):
        text = str(text)
    # Then proceed with normalization
    return re.sub(r'[^A-Z0-9 \.]', '', text.upper())

# Function to extract Tata model
def extract_tata_model(text):
    txt = normalize(text)
    
    # Skip entries that clearly aren't Tata vehicles
    non_tata = ["MARUTI", "MAHINDRA", "HONDA", "HYUNDAI", "FORD", "TOYOTA", "ASHOK", "NON TATA"]
    for brand in non_tata:
        if brand in txt:
            return "NOT FOUND"
    
    # Check for Jaguar Land Rover (owned by Tata but treated separately)
    if any(luxury in txt for luxury in ["JAGUAR", "LAND ROVER", "RANGE ROVER", "DISCOVERY"]):
        return "JAGUAR LAND ROVER"
    
    # First check for specific model combinations
    specific_models = [
        "INDICA V2", "INDIGO MARINA", "INDIGO CS", "INDIGO ECS", "INDIGO MANZA",
        "SAFARI STORME", "SAFARI DICOR", "SUMO GOLD", "SUMO VICTA", "SUMO GRANDE",
        "TIAGO NRG", "NEXON EV", "ACE GOLD", "ACE MEGA", "ACE ZIP", "ACE MAGIC", 
        "SUPER ACE", "XENON XT"
    ]
    
    for model in specific_models:
        if model in txt:
            return model
    
    # Check for commercial vehicle model combinations (truck codes)
    truck_patterns = [
        r'LPT\s?1109', r'LPT\s?1512', r'LPT\s?1613', r'LPT\s?1616', 
        r'LPT\s?2515', r'LPT\s?2518', r'LPT\s?3118', r'LPT\s?3518', 
        r'LPT\s?4018', r'LPT\s?4923', r'LPT\s?407', r'LPT\s?709',
        r'SFC\s?407', r'SFC\s?709', r'SK\s?407', r'SK\s?1613',
        r'LP[TKS]?\s?4923', r'SIGNA\s?4923', r'PRIMA\s?4018'
    ]
    
    for pattern in truck_patterns:
        if re.search(pattern, txt):
            match = re.search(pattern, txt).group(0)
            # Clean up any spaces in the match
            match = re.sub(r'\s+', ' ', match).strip()
            return match
    
    # Fix aliases (correct misspellings and other variations)
    for bad, good in tata_alias_map.items():
        if bad in txt:
            if good == "NON TATA" or good == "NOT TATA":
                return "NOT FOUND"
            if good == "JAGUAR LAND ROVER":
                return "JAGUAR LAND ROVER"
            txt = txt.replace(bad, good)
    
    # Check for base passenger car models
    passenger_models = [
        "INDICA", "VISTA", "BOLT", "TIAGO", "ALTROZ", "NANO", "INDIGO", 
        "MANZA", "TIGOR", "ZEST", "SAFARI", "NEXON", "HARRIER", "SUMO", 
        "ARIA", "HEXA", "PUNCH"
    ]
    
    for model in sorted(passenger_models, key=len, reverse=True):
        if model in txt:
            # For popular models, check for trim levels
            trim_levels = ["XZ+", "XZA+", "XZ", "XZA", "XT", "XTA", "XM", "XMA", "XE"]
            for trim in trim_levels:
                if trim in txt:
                    return f"{model} {trim}"
            return model
    
    # Check for small commercial vehicles
    cv_models = ["ACE", "INTRA", "MAGIC", "WINGER", "VENTURE", "IRIS", "XENON", "YODHA", "ZIP"]
    for model in sorted(cv_models, key=len, reverse=True):
        if model in txt:
            return model
    
    # Check for truck model series
    truck_models = ["LPT", "LPS", "LPK", "LPO", "SIGNA", "PRIMA", "ULTRA"]
    for model in truck_models:
        if model in txt:
            # Look for common numeric codes that follow truck models
            for code in ["407", "709", "909", "1109", "1512", "1613", "2515", "2518", "3118", "3518", "4018"]:
                if code in txt:
                    return f"{model} {code}"
            return model
    
    # Check for any other keywords from our list
    for keyword in sorted(tata_model_keywords, key=len, reverse=True):
        if keyword in txt:
            return keyword
    
    # If nothing matches, use fuzzy matching as fallback
    try:
        # Use rapidfuzz for fuzzy matching
        from rapidfuzz import process
        
        # Try to find the best match with a minimum score
        result = process.extractOne(txt, tata_model_keywords, scorer=fuzz.token_set_ratio)
        if result and result[1] >= 80:  # if similarity score is at least 80%
            return result[0]
    except Exception:
        # If rapidfuzz isn't available or fails, we'll just continue
        pass
    
    return "NOT FOUND"

# Apply the extraction to the dataset
tata_data['tata_model'] = tata_data['rc_maker_model'].apply(extract_tata_model)

# Count the frequency of each extracted model
model_distribution = tata_data['tata_model'].value_counts()
print("\nExtracted model distribution:")
print(model_distribution.head(20))

# Create final model column with fallback logic
tata_data['final_model'] = tata_data['tata_model']
tata_data.loc[tata_data['final_model'] == "NOT FOUND", 'final_model'] = "TATA OTHER"

# Save the processed data
tata_data.to_csv('/home/bipin/Documents/kotak/car_model/clean_model/tata_processed.csv', index=False)

# Save the model mapping for verification and record
model_mapping = pd.DataFrame({
    'original_model': tata_data['rc_maker_model'].unique()
})
model_mapping['cleaned_model'] = model_mapping['original_model'].apply(clean_model)
model_mapping['extracted_model'] = model_mapping['original_model'].apply(extract_tata_model)
model_mapping['final_model'] = model_mapping['extracted_model']
model_mapping.loc[model_mapping['final_model'] == "NOT FOUND", 'final_model'] = "TATA OTHER"

model_mapping.to_csv('/home/bipin/Documents/kotak/car_model/clean_model/tata_model_mapping.csv', index=False)

print("\nProcessing complete. Files saved:")
print("1. Processed data: tata_processed.csv")
print("2. Model mapping: tata_model_mapping.csv")

# Print some statistics
print(f"\nTotal records: {len(tata_data)}")
print(f"Unique original models: {len(tata_data['rc_maker_model'].unique())}")
print(f"Unique extracted models: {len(tata_data['tata_model'].unique())}")
print(f"Records mapped to known models: {(tata_data['tata_model'] != 'NOT FOUND').sum()}")
print(f"Records not mapped (TATA OTHER): {(tata_data['tata_model'] == 'NOT FOUND').sum()}")
print(f"Mapping success rate: {(tata_data['tata_model'] != 'NOT FOUND').sum() / len(tata_data) * 100:.2f}%")